# Function to fit our analytical model to the SF-gas from TNG

Our fit procedure is also described in section 3 of van son et al in prep.




In [1]:
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

from scipy import interpolate
from scipy.optimize import minimize

# from astropy.cosmology import WMAP9, z_at_value
from astropy.cosmology import Planck18  as cosmo# Planck 2018
from astropy.cosmology import z_at_value

############################
# Custom scripts
import get_ZdepSFRD as Z_SFRD
import importlib
importlib.reload(Z_SFRD)

Cosmol_sim_location = '/Users/lieke/surfdrive/Documents/CompareCOMPAS/' + "SFRMetallicityFromGasTNG100.hdf5"


# First read the TNG data 
## And convert it to a SFDR in Msun/yr/Mpc^-3

***
We will have simulation data == TNG in our case
and model data  == our analytical function
***


In [4]:
##########################################
# Simulated SFRD data (from TNG)
##########################################
## Read in the pure simulation data
with h5.File(Cosmol_sim_location, "r") as f:
    MetalBins     = f["MetalBins"][:]
    Lookbacktimes = f["Lookbacktimes"][:]
    BoxSfr        = f["Sfr"][:]
# Convert SFR from sfr/box to sfr Mpc-3
littleh  = 0.6774
Rbox     = 75/littleh
Sim_SFRD = BoxSfr / Rbox**3 *u.Mpc**-3
Sim_SFRD = Sim_SFRD.value

## Adjust what metallicities to include 
# The minimum metallicity in COMPAS is 10^-4, so there is no
# use in fitting to the 10^-7 metallicity tail
Sim_center_Zbin = (MetalBins[:-1] + MetalBins[1:])/2.
low_bound_Z_ind  = np.where(Sim_center_Zbin > 1e-5)[0]# index of Sim_center_Zbin, where Z > 1e-5
tofit_Sim_metals = Sim_center_Zbin[low_bound_Z_ind]   

## Reverse the time axis of the SFRD and lookback time for the fit
tofit_Sim_SFRD      = Sim_SFRD[:,low_bound_Z_ind][::-1]
tofit_Sim_lookbackt = Lookbacktimes[::-1] 

## Convert lookback times to redshifts
# the last value of Lookbacktimes = 0, which is problematic for z calculation
redshifts_Sim = [z_at_value(cosmo.lookback_time,t*u.Gyr) for t in Lookbacktimes[:-1]] 
redshifts_Sim.append(0) # put redshift zero back at the end
redshifts_Sim = np.array(redshifts_Sim)


# Interpolate the TNG data

Using scipy interpolate
https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html

f = interpolate.interp2d(x, y, z)



In [5]:
#########################################
# Interpolate the simulation data
f_interp = interpolate.interp2d(tofit_Sim_lookbackt, tofit_Sim_metals, tofit_Sim_SFRD.T, kind='cubic')

#########################################
# Retrieve values at higher res regular intervals
redshift_new         = np.arange(0, 10.1, 0.05)
Lookbacktimes_new    = [cosmo.lookback_time(z).value for z in redshift_new]

log_tofit_Sim_metals =  np.log10(tofit_Sim_metals)
metals_new           = np.logspace(min(log_tofit_Sim_metals), max(log_tofit_Sim_metals), 50)

SFRDnew = f_interp(Lookbacktimes_new,metals_new)

print('np.shape(Lookbacktimes_new)', np.shape(Lookbacktimes_new),'np.shape(metals_new)',
      np.shape(metals_new), 'np.shape(SFRDnew)', np.shape(SFRDnew) )


np.shape(Lookbacktimes_new) (202,) np.shape(metals_new) (50,) np.shape(SFRDnew) (50, 202)


In [6]:

##################################################
# muz =-0.09, mu0 =0.026, sigma =1.9, alpha=-3.3
##################################################
def calc_chi_square(fit_metals, Redshifts = [],  simulation_SFRD = [],
                    mu_0_list = 0.026, muz_list =-0.09, sigma0_list = 1.9, sigmaz_list = 1.9, alpha_list =-3.3,
                    sf_a =0.01 , sf_b=2.6, sf_c=3.2 , sf_d=6.2):
    """
    Calculate the distribution of metallicities at different redshifts using a log skew normal distribution
    that is basically a skew normal distribution, but then with the random variable x = ln(Z)

    NOTE: This assumes that metallicities in COMPAS are drawn from a flat in log distribution

    Args:
        fit_metals              --> [float]          metals used for fitt
        Redshifts               --> [float]          redshihts used to fit
        simulation_SFRD           --> [float]        cosmological simulation SFRD to fit to
        
        mu_0_list    = -0.23    --> [float]          location (mean in normal) at redshift 0
        muz_list = 0.035    --> [float]          redshift evolution of the location
        sigma0_list  = 0.39     --> [float]          Scale at redshift 0 (variance in normal)
        sigmaz_list  = 0.0      --> [float]          redshift evolution of Scale (variance in normal)
        alpha_list   = 0.0      --> [float]          shape (skewness, alpha = 0 retrieves normal dist)

        sf_a                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_b                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_c                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)
        sf_d                    --> [float]          SFR(z) parameter (shape of Madau & Dickenson 2014)

    Returns:
        tot_chi_square          --> [float ] 
    """
    
    """
    ######################################

    """  
    print('muz_list', muz_list, 'mu_0_list',mu_0_list, 'sigma0_list',sigma0_list,'sigmaz_list',sigmaz_list, 'alpha_list',alpha_list,
          'sf_a', sf_a, 'sf_b', sf_b, 'sf_c', sf_c, 'sf_d', sf_d)
        
    #####################################
    # Get the SFR
    # Madau & Fragos 2017: a=0.01, b=2.6, c=3.2,  d=6.2
    sfr = Z_SFRD.Madau_Dickinson2014(Redshifts, a=sf_a, b=sf_b, c=sf_c, d=sf_d) # Msun year-1 Mpc-3 

    # Get dPdZ 
    dPdlogZ, redshifts, metallicities, step_logZ, p_draw_metallicity = \
                    Z_SFRD.skew_metallicity_distribution(mu_z = muz_list, mu_0 = mu_0_list,
                                                  sigma_0= sigma0_list, sigma_z=sigmaz_list, alpha = alpha_list, 
                                                  metals=fit_metals, redsh = Redshifts)
        
    log_fit_metallicities = np.log(fit_metals)
    step_fit_logZ         = np.diff(log_fit_metallicities)[0]
    
    ######################################
    # For each redshift in the TNG data:
    tot_chi_square = 0
        
    for redshift_i in range(len(redshifts)):
        #print(redshift_i, 'at redshift', redshifts[redshift_i])
        ######################################
        # Now the SFRD = sfr x dPdZ
        SFRD = sfr[redshift_i] *dPdlogZ[redshift_i,:] # Msun year-1 Mpc-3 

        # Model comes in dP/dlogZ, so should your sim-data 
        obs   = simulation_SFRD[redshift_i,:]/step_fit_logZ  # 
        model = SFRD.value 
        
        ###################
        # Zero values are troublesome in Chi_squared!
        # only fit where obs > x 
        common_bool = np.logical_and(model > 1e-11, obs > 1e-11)
        model       = model[common_bool]
        obs         = obs[common_bool]

        ###################
        # Actual CHI_squared
        chi_square = ((obs - model )**2)/np.sum(model) 
        
        tot_chi_square += np.sum(chi_square, axis = -1)


    ######################################
    # Minimum Chi_squared taking all redshift into aaccount
    print(tot_chi_square)
    
    return tot_chi_square


    
    

# Run your chi square calculations

## and leave the refinement up to scipy minimize

In [7]:
#################################################################
## Function wrapper to minimize the Chi_square
#################################################################
def test_chi(x0 = [-0.09, 0.026, 1.9, 0.1, -3.3,0.01, 2.6, 3.2, 6.2] ):
    chi_square_matix = calc_chi_square(metals_new, Redshifts = redshift_new, simulation_SFRD = SFRDnew.T, 
                                       muz_list =x0[0], mu_0_list =x0[1],sigma0_list =x0[2], sigmaz_list=x0[3], alpha_list =x0[4],
                                       sf_a =x0[5], sf_b=x0[6], sf_c=x0[7], sf_d=x0[8])
    
    return chi_square_matix


# BEST GUESS
x0 = np.array([-0.15, 0.026, 1.1, 0.1, -3.3, 0.01, 2.6, 3.2, 6.2])
# FIT
res = minimize(test_chi, x0= x0, method ='nelder-mead', options = {'maxiter': 5000})




muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
step_logZ 0.22712574046539835
2.2979603921969303
muz_list -0.1575 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
step_logZ 0.22712574046539835
2.3509401460454034
muz_list -0.15 mu_0_list 0.0273 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
step_logZ 0.22712574046539835
2.2583347959062263
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1550000000000002 sigmaz_list 0.1 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
step_logZ 0.22712574046539835
2.36561052359096
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.10500000000000001 alpha_list -3.3 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
step_logZ 0.22712574046539835
2.3456031984629937
muz_list -0.15 mu_0_list 0.026 sigma0_list 1.1 sigmaz_list 0.1 alpha_list -3.465 sf_a 0.01 sf_b 2.6 sf_c 3.2 sf_d 6.2
step_logZ 0.2271257404653

/Users/lieke/surfdrive/Documents/SFRD_fit/src/scripts/get_ZdepSFRD.py:107: RuntimeWarning: divide by zero encountered in true_divide
  mu_metallicities = np.log(mean_metallicities/(2.*PHI) * 1./(np.exp(0.5*sigma**2) )  )
/Users/lieke/surfdrive/Documents/SFRD_fit/src/scripts/get_ZdepSFRD.py:129: RuntimeWarning: invalid value encountered in true_divide
  dPdlogZ = dPdlogZ /norm[:,np.newaxis]


1.5481098755183567
muz_list -0.14933927754915408 mu_0_list 0.026527450083828678 sigma0_list 1.0891190367322046 sigmaz_list 0.0959175430574607 alpha_list -3.3669455875628724 sf_a 0.010202865416857184 sf_b 2.6527450083828694 sf_c 3.2649169333942982 sf_d 6.09709190672154
step_logZ 0.22712574046539835
1.4059719614691033
muz_list -0.1490089163237311 mu_0_list 0.02679117512574302 sigma0_list 1.0836785550983068 sigmaz_list 0.09137631458619105 alpha_list -3.400418381344309 sf_a 0.010304298125285775 sf_b 2.6791175125743036 sf_c 3.297375400091447 sf_d 6.04563786008231
step_logZ 0.22712574046539835
1.0659725226247927
muz_list -0.14911903673220542 mu_0_list 0.026703266778438244 sigma0_list 1.0854920489762732 sigmaz_list 0.09900116852105875 alpha_list -3.3892607834171633 sf_a 0.010270487222476246 sf_b 2.670326677843826 sf_c 3.286555911192398 sf_d 6.0627892089620525
step_logZ 0.22712574046539835
1.2204188973563592
muz_list -0.14892326711713996 mu_0_list 0.02685954828475785 sigma0_list 1.082268059859

0.5632304371816285
muz_list -0.1392717968600429 mu_0_list 0.02790373503897572 sigma0_list 1.0370216683715254 sigmaz_list 0.08847918456304688 alpha_list -3.431221160049801 sf_a 0.010566904393823047 sf_b 2.7696184016302907 sf_c 3.4489596282252917 sf_d 6.12363276232342
step_logZ 0.22712574046539835
0.48623465596520965
muz_list -0.13536662177697445 mu_0_list 0.027680957835102403 sigma0_list 1.0458607863564826 sigmaz_list 0.08836368838962802 alpha_list -3.343105698048559 sf_a 0.010613356066124645 sf_b 2.8187908683334726 sf_c 3.5289143111565973 sf_d 6.039367476469667
step_logZ 0.22712574046539835
0.47438306367071476
muz_list -0.13948762231779893 mu_0_list 0.02701230214416718 sigma0_list 1.0405297403516989 sigmaz_list 0.08445315182842184 alpha_list -3.4218021345887837 sf_a 0.010821488576706114 sf_b 2.808188406552751 sf_c 3.5744187470603235 sf_d 6.012609912495002
step_logZ 0.22712574046539835
0.5537264961915087
muz_list -0.1350546475168826 mu_0_list 0.0277527145119507 sigma0_list 1.03507152756

0.35574011960400986
muz_list -0.12312498189248434 mu_0_list 0.03158703328378002 sigma0_list 1.0572633557945095 sigmaz_list 0.07418256287663523 alpha_list -3.2618681311240687 sf_a 0.010392208039860445 sf_b 2.7492906967344597 sf_c 3.623032258636753 sf_d 6.364210168225652
step_logZ 0.22712574046539835
0.33011854105124583
muz_list -0.12425932388612676 mu_0_list 0.031215379215809497 sigma0_list 1.0482663879140506 sigmaz_list 0.07517553371127944 alpha_list -3.38505458197403 sf_a 0.010547229836640503 sf_b 2.7725405991543437 sf_c 3.546690448540917 sf_d 6.027461700924726
step_logZ 0.22712574046539835
0.33559807814515974
muz_list -0.1114998727368246 mu_0_list 0.032881082977065726 sigma0_list 1.0588332083138954 sigmaz_list 0.06677736087114361 alpha_list -3.330491633603949 sf_a 0.010432964566809536 sf_b 2.744171622621537 sf_c 3.7060096540955083 sf_d 6.569152601952915
step_logZ 0.22712574046539835
0.3633915165982124
muz_list -0.12700597096587088 mu_0_list 0.03198838191808409 sigma0_list 1.053706992

...

0.05983922714388216
muz_list -0.04770758167034118 mu_0_list 0.024831303498084427 sigma0_list 1.1187890882864724 sigmaz_list 0.04774284320615513 alpha_list -1.7095547174238366 sf_a 0.028487616584414663 sf_b 1.4817225723304996 sf_c 4.4531482904697395 sf_d 5.893633414012406
step_logZ 0.22712574046539835
0.059839179341646384
muz_list -0.04768023801097891 mu_0_list 0.024825601791037037 sigma0_list 1.1183461946285822 sigmaz_list 0.04778471741393589 alpha_list -1.7091676481351645 sf_a 0.028485803349736223 sf_b 1.4815552575253934 sf_c 4.4535700381523124 sf_d 5.89375942999117
step_logZ 0.22712574046539835
0.05983917653126654
muz_list -0.047690130480576944 mu_0_list 0.024831672916433695 sigma0_list 1.1184348281833145 sigmaz_list 0.04776979944874678 alpha_list -1.7083770173484791 sf_a 0.028487149351228848 sf_b 1.4817747429158408 sf_c 4.45307908188494 sf_d 5.893148485332603
step_logZ 0.22712574046539835
0.059839148384344974
muz_list -0.04773395167289776 mu_0_list 0.024828651629311846 sigma0_list 1

0.05983906347990911
muz_list -0.047693664434783595 mu_0_list 0.024837056059367035 sigma0_list 1.1183574158251148 sigmaz_list 0.04777770342702972 alpha_list -1.7082678609915756 sf_a 0.02848168048891095 sf_b 1.4820511910635457 sf_c 4.452461612368807 sf_d 5.892285255695683
step_logZ 0.22712574046539835
0.05983909183189625
muz_list -0.04769761803648932 mu_0_list 0.02483547020725797 sigma0_list 1.1183326743466073 sigmaz_list 0.04777531386451908 alpha_list -1.7083615020913545 sf_a 0.02848414701735286 sf_b 1.4819254882976653 sf_c 4.45261887005444 sf_d 5.892444681584027
step_logZ 0.22712574046539835
0.05983906323503484
muz_list -0.047694892279263185 mu_0_list 0.02483509804117088 sigma0_list 1.1182444724135 sigmaz_list 0.047766185004339545 alpha_list -1.7079419617087732 sf_a 0.02848104763670268 sf_b 1.4820294719740494 sf_c 4.45246948211617 sf_d 5.892514587898212
step_logZ 0.22712574046539835
0.05983904708684317
muz_list -0.04768917955311655 mu_0_list 0.024836057223726393 sigma0_list 1.118184531

0.05983903179105053
muz_list -0.047709140799897916 mu_0_list 0.024838524567043635 sigma0_list 1.1182743040935423 sigmaz_list 0.047765423066039016 alpha_list -1.7078399095781074 sf_a 0.028487197425107066 sf_b 1.4817997302500303 sf_c 4.452676714711682 sf_d 5.89254456207672
step_logZ 0.22712574046539835
0.05983901969445226
muz_list -0.04770696189789757 mu_0_list 0.0248385194743503 sigma0_list 1.118285607565186 sigmaz_list 0.04776399647730871 alpha_list -1.7078282329800523 sf_a 0.02848853570974552 sf_b 1.4817262079446083 sf_c 4.452759857740938 sf_d 5.892610619558923
step_logZ 0.22712574046539835
0.05983901129775375
muz_list -0.047711666435060834 mu_0_list 0.02483993675010256 sigma0_list 1.1182396906035685 sigmaz_list 0.04776088593011558 alpha_list -1.7076668835543736 sf_a 0.028485286689372516 sf_b 1.4819268350067247 sf_c 4.4524243060181945 sf_d 5.892308595339166
step_logZ 0.22712574046539835
0.05983903173143217
muz_list -0.0477234446189861 mu_0_list 0.0248387580173335 sigma0_list 1.1183523

0.05983898390853614
muz_list -0.047704035212938325 mu_0_list 0.024839293929531353 sigma0_list 1.118104639481471 sigmaz_list 0.047758267133724414 alpha_list -1.7072276590963593 sf_a 0.028497652882005925 sf_b 1.4812269541515573 sf_c 4.453471203585362 sf_d 5.893361589417542
step_logZ 0.22712574046539835
0.05983898143690171
muz_list -0.04770676805775899 mu_0_list 0.024842034705815485 sigma0_list 1.1181851380765728 sigmaz_list 0.047754007660024536 alpha_list -1.7069355257247785 sf_a 0.028497381755915943 sf_b 1.4813400083752164 sf_c 4.453303204716928 sf_d 5.893192235403362
step_logZ 0.22712574046539835
0.0598389847927579
muz_list -0.047717756830659167 mu_0_list 0.024840726162427197 sigma0_list 1.1180871809291322 sigmaz_list 0.04775091795417831 alpha_list -1.707014558272535 sf_a 0.028493952642840693 sf_b 1.4814503448249354 sf_c 4.453088170831602 sf_d 5.893097187268317
step_logZ 0.22712574046539835
0.0598389856430861
muz_list -0.04771268581370315 mu_0_list 0.024841825241772978 sigma0_list 1.11

0.05983897542036973
muz_list -0.04770299159354703 mu_0_list 0.024840451485317855 sigma0_list 1.1180431045246828 sigmaz_list 0.04775885915938516 alpha_list -1.7068224002783055 sf_a 0.02849520590947514 sf_b 1.48131804334123 sf_c 4.453363003692582 sf_d 5.893328870415853
step_logZ 0.22712574046539835
0.05983897343860213
muz_list -0.04770470330917366 mu_0_list 0.024841698007867823 sigma0_list 1.118097867116452 sigmaz_list 0.047756026662805345 alpha_list -1.706774141868991 sf_a 0.028495756268376658 sf_b 1.481348970283121 sf_c 4.453316575847491 sf_d 5.893276429981267
step_logZ 0.22712574046539835
0.059838975086318136
muz_list -0.04771380704364401 mu_0_list 0.024842595006610117 sigma0_list 1.118084667207583 sigmaz_list 0.04775589925875884 alpha_list -1.7068215223473886 sf_a 0.028494081292542724 sf_b 1.4813906026353518 sf_c 4.4532576128882155 sf_d 5.893301541781254
step_logZ 0.22712574046539835
0.0598389742144125
muz_list -0.04771337417074677 mu_0_list 0.024841239634990624 sigma0_list 1.1181085

In [8]:
print(res.success, res.message, res.nit)
muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best = res.x[0], res.x[1], res.x[2], res.x[3],res.x[4]
sf_a_best, sf_b_best, sf_c_best, sf_d_best    = res.x[5], res.x[6], res.x[7], res.x[8] 

print('\nBEST FITTING PARAMETERS:')
print('muz =%s, mu0 =%s, sigma_0 =%s, sigma_z =%s, alpha=%s'% (muz_best, mu0_best, sigma0_best, sigmaz_best, alpha_best) )
print('sf_a =%s, sf_b =%s, sf_c =%s, sf_d =%s'% (sf_a_best, sf_b_best, sf_c_best, sf_d_best) )


True Optimization terminated successfully. 2439

BEST FITTING PARAMETERS:
muz =-0.04770354384519166, mu0 =0.02484072908483711, sigma_0 =1.1180662055781732, sigma_z =0.047758870831842756, alpha=-1.7068556943711979
sf_a =0.028495969925032248, sf_b =1.48131212310957, sf_c =4.453336741929566, sf_d =5.893236318366001
